In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_stata("../data/gss2020panel_r1a.dta", convert_categoricals=False)

C:\Users\yuanx\AppData\Local\Temp\ipykernel_22156\1295668150.py:1: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata("../data/gss2020panel_r1a.dta", convert_categoricals=False)


In [4]:
# select 2028 data
df_2018 = df[df['samptype'] == 2018].copy()
# select variables
col_list = ['marital_1b', 'mntlhlth_1b', 'degree_1b', 'satfin_1b', 'neisafe_1b', 'relpersn_1b', 'race_1b']
df_2018_sub = df_2018[col_list].copy()
# drop rows that have any NA
df_2018_sub = df_2018_sub.dropna(how='any')
# only keep married (marital == 1) and never married (marital == 5)
df_2018_sub.loc[df_2018_sub['marital_1b'].isin([1,5])].shape
df_2018_sub.shape
# rename columns to remove suffix
col_dict = {x: x.split('_')[0] for x in list(df_2018_sub.columns)}
df_2018_sub.rename(columns=col_dict, inplace=True)
df_2018_sub['marital'] = df_2018_sub['marital'].apply(lambda x: 1 if x == 1 else 0)
# one-hot encoding race variable
df_2018_sub['race_is_white'] = df_2018_sub['race'].isin([1])
df_2018_sub['race_is_black'] = df_2018_sub['race'].isin([2])
df_2018_sub['race_is_other'] = df_2018_sub['race'].isin([3])
df_2018_sub.drop(columns=['race'], inplace=True)
df_2018_sub.head()

,marital,mntlhlth,degree,satfin,neisafe,relpersn,race_is_white,race_is_black,race_is_other
2867,0,20.0,2.0,3.0,1.0,1.0,True,False,False
2869,1,3.0,3.0,2.0,1.0,3.0,True,False,False
2870,1,1.0,3.0,1.0,1.0,3.0,True,False,False
2873,0,0.0,1.0,2.0,3.0,2.0,False,True,False
2874,0,1.0,1.0,2.0,2.0,2.0,True,False,False


In [5]:
# train test split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X = df_2018_sub.drop(columns=['mntlhlth', 'marital'])
y = df_2018_sub['marital']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

In [6]:
# random search the hyperparameter space
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [7]:
rf = RandomForestClassifier(random_state=99)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
print(rf_random.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [ ]:
# grid search near the hyperparameters obtained through random search
from sklearn.model_selection import GridSearchCV
param_grid = {
    'bootstrap': [True],
    'max_depth': [50,70,90],
    'max_features': ['log2'],
    'min_samples_leaf': [2,4,6],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [200,400,600]
}
rf = RandomForestClassifier(random_state=99)
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, verbose = 2)
grid_search.fit(X_train, y_train)

In [ ]:
# print best parameters
grid_search.best_params_

In [ ]:
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

In [ ]:
print('Test set score:', best_rf.score(X_test, y_test))
print('Train set score:', best_rf.score(X_train, y_train))

In [ ]:
df_2018_sub.head()

In [ ]:
# plt.hist(df_2018_sub["propensity"], bins=20, alpha=0.3, label="All")
plt.hist(df_2018_sub.query("marital==0")["propensity"], bins=20, alpha=0.3, color="C2", label='Never Married')
plt.hist(df_2018_sub.query("marital==1")["propensity"], bins=20, alpha=0.3, color="C3", label=' Married')
plt.legend()

In [ ]:
df_2018_sub.to_csv(path_or_buf='data_2018.csv', index=False)